This Notebook contains my quest to get genres out of Goodreads.
First, we set up the webscraping.

In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import firefox
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service

In [2]:
#Fill in my Goodreads username and password from the .env
user_name = os.environ.get('USER')
password = os.environ.get('PASSWORD')

#This is just the URL I get when I go to goodreads and select log in by email.
login_url = os.environ.get('URL')

In [3]:
#Here I start up a headless Firefox browser through Selenium
s = Service("geckodriver.exe")
opts=Options()
opts.add_argument('-headless')
browser = webdriver.Firefox(service=s)
browser.get(login_url)

In [4]:
# Here I log into goodreads
log_email = browser.find_element(By.ID, value="ap_email")
log_pwd = browser.find_element(By.ID, value="ap_password")
log_email.send_keys(user_name)
log_pwd.send_keys(password)
log_pwd.submit()

This is where things get shaky. I'm able to make this work for individual titles, but as soon as I put it into the function, the itemqueue ends up an empty list. Sometimes even when it's not in a function, it's an empty list. I haven't quite been able to figure out what triggers it to actually work. 

In [7]:
title = "The Thief"
browser.get('http://www.goodreads.com/search?q=&qid=')
search_book = browser.find_element(By.ID, value='search_query_main')
search_book.send_keys(title)
search_book.submit()

In [8]:
itemqueue = browser.find_elements(By.XPATH, value="//table/tbody/tr[contains(@itemtype, 'http://schema.org/Book')]")
img = browser.find_elements(By.CLASS_NAME, value="bookCover")
book_list = list()
for i in range(len(itemqueue)):
    book_list.append(itemqueue[i].text.split('\n'))
    book_list_ap = list()
for i in range(0, len(book_list)):
     book_list_ap.append((book_list[i][0],book_list[i][1],img[i].get_property("src")))

In [10]:
for book in book_list_ap:
    if "by Megan Whalen Turner" in book[1]:
        book_id = book[2].split('/')[-1].split('.')[0]
        break
book_url = f'https://www.goodreads.com/book/show/{book_id}'
book_url

'https://www.goodreads.com/book/show/448873'

In [ ]:
book_list_ap

In [ ]:
find_book_1("1984")

In [ ]:
find_item_queue()

In [ ]:
def search_for_title(title):
    browser.get('http://www.goodreads.com/search?q=&qid=')
    search_book = browser.find_element(By.ID, value='search_query_main')
    search_book.send_keys(title)
    search_book.submit()

In [ ]:
def find_book(title, author):
    # Finds book in the title that's by the author
    itemqueue = browser.find_elements(By.XPATH, value="//table/tbody/tr[contains(@itemtype, 'http://schema.org/Book')]")
    img = browser.find_elements(By.CLASS_NAME, value="bookCover")
    book_list = list()
    for i in range(len(itemqueue)):
        info = itemqueue[i].text.split('\n')
        if 'by ' + author in itemqueue[i][1]:
            break
        for i in range(0, len(book_list)):
            book_list_ap.append((book_list[i][0],book_list[i][1],book_list[i][2],img[i].get_property("src")))
    return len(itemqueue)

In [ ]:
search_for_title("1984")

In [ ]:
find_book("1984", "George Orwell")

In [ ]:
len(itemqueue)

In [ ]:
#browser.quit()